# CS153 Assignment 6
## Deep Learning Practice

In the sixth assignment for CS153 in Fall 2024 we are going to get further practice with deep learning as it relates to the field of computer vision. You will be responsible for producing several outputs: code to allow for transfer learning over pre-trained neural networks, saved files produced by your code, and written responses to questions related to your network output. 

Rather than a PDF description of the assignment, we are trying an alternative method of assignment dissemination through a Jupyter Notebook (this notebook!). _You should not need alter the contents of this notebook_. Please feel free to make a sandbox copy to aid in your development, but the code you turn in should work with an unmodified version of this notebook.

Your code will modify the provided file `A6.py`, and which you will turn in on Gradescope to the "Assignment 6 Code" entry. Written questions, while described here, will be turned in directly on Gradescope through the "Assignment 6 Written" entry, along with the uploads of the two images created by this notebook (which will be placed in an `output` folder in your current working directory).

## Question 1: Transfer Learning
\[10 points code, 14 points written answers\]




The flow of this question will follow the same flow that had in class for our transfer learning demonstration, namely: first data handling, then model handling, then training setup, and finally execution, only this time our code will be encapsulated within the imported `A6` code from your `A6.py` file. Within the starting file you are provided with function stubs and partially implemented solutions, and each section of this notebook will provide instructions for what you are expected to do to complete the function being called. You should execute cells in order; some cells will simply provide supporting computations, while others will have tasks for you to complete in `A6.py` to make the cell run as expected. For any cell that has tasks you must complete, it will include a section titled in bold, "**Your Task**".

This first cell will import libraries and run some setup routines for this notebook. You do not have any tasks associated with this cell. While this notebook can run on CPU, it is highly recommended to run it on one of the available GPU servers. 

Because there are a number of students in this course with many of you potentially working at the same time, before you run your code you should you should connect to the server and check to see whether there are sufficient resources available on the GPU. You can do this by running the command `nvidia-smi` in the terminal. This will print out a summary of the GPU status, including the currently utilized memory and the current jobs running on the device. Please feel free to discuss on Slack the usage of GPUs (in case someone has set up a notebook that is utilizing memory and did not end up relinquishing it even though their job is done).

In [ ]:
# Most of the functionality will take place in your .py file, but we need a few libraries here
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as tutils
import A6
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Let's set up data handling. You do not have a programming task for this section, but you _should look at the code and make sure you understand it_, because you will be using the dataloader in your training routine and will be asked questions about it in the written portion.

This section will depend on a custom dataset. This is too large (~370MB) to host on Sakai, so the dataset is stored on the CS servers at `/cs/cs153/datasets/a6data`. You can create your own local copy via `scp`, or you can directly load the data from this location if you are working on the servers.

You will be making two different versions of the dataloader; these are different based on the data transforms that they encode. Note that the `datadict` does not change between the versions, so we only need one copy.

In [ ]:
data_dir = '/cs/cs153/datasets/a6data' # this line assumes you are working from the CS servers.
# data_dir = 'a6data' # Uncomment and use this line if using a copy local to your current working directory.

dataloader_v1, datadict = A6.build_dataloader(data_dir, version = 0)

dataloader_v2, _ = A6.build_dataloader(data_dir, version = 1)

We can use our helper functions to visualize the output of the dataloaders. This may be helpful to better intuitively understand the difference between them.

In [ ]:
# Visualize Version 1

inputs, classes = next(iter(dataloader_v1['train']))

out = tutils.make_grid(inputs)

A6.tensor_show(out, title=[datadict["class_names"][x] for x in classes])

In [ ]:
# Visualize Version 2

inputs, classes = next(iter(dataloader_v2['train']))

out = tutils.make_grid(inputs)

A6.tensor_show(out, title=[datadict["class_names"][x] for x in classes])

Let's set up our training model. For this assignment, we'll be doing transfer learning using a pre-trained VGG16 model. However, since VGG is rather top-heavy (and also to make sure we have slightly more interesting training curves), we're going to turn off one of the fully-connected layers (and its accompanying ReLu and dropout layers) and instead replace those layers with identity pass-throughs. This code has already been implemented for you.

**Your Task**: \[6 points\]
You need to modify the code in `A6.py` at the point that it says `TODO` and set up the final layer of the network such that it is able to be applied to our custom dataset.

Some potentially useful hints:
- Remember that you can call `print(model)` to see the available labels within a model architecture in order to modify particular components.
- The final output layer is going to be a `Linear` layer (the PyTorch term for a fully connected layer), which needs to be specified by two arguments: the number of inputs, and the number of outputs.
- When a model like VGG (which has a simple feedforward structure) has multiple numerically tagged layers under a common handle, you can access a specific single layer using the syntax `model.HANDLE[LAYER]`. So, for example, the first convolution layer would be accessed by `model.features[0]`

Note: in this code block we will make two independent copies of our model. This will allow us to train one with each version of our dataloader.

In [ ]:
model1_dict = A6.setup_model(datadict, device)

model2_dict = A6.setup_model(datadict, device)

As a sanity check, let's send some data through the network to make sure we get predictions and not errors. Since our models are identical at this point, we'll just run `model1`. 

**Written Question 1: Initial Network Prediction**: \[2 points\] 
Do you expect this initial set of predictions to be accurate? Why or why not?

In [ ]:
A6.visualize_model(model1_dict["model"], dataloader_v1, datadict["class_names"], device)

This next section will train our two versions of the model with `train_model` using the two different dataloaders we set up earlier. The training procedure is already provided for you, but is currently set up to return an empty list for the `classlog`.

**Your Task**: \[9 points\]
Add code to the `train_model` so that `classlog` contains the class-specific validation accuracy for every epoch. This should be in the form of a list of length `num_epochs`, and each entry of the list is a `numpy` array of length `num_classes`. Each element of this `numpy` array is the ratio of true positive predictions for that class to the total number of validation samples for that class (in our dataset, all classes have the same number of validation samples, 80, which is already computed in the `numval` variable in your `train_model` function).

In [ ]:
model1_dict, log1 = A6.train_model(model1_dict, dataloader_v1, datadict, device)

In [ ]:
model2_dict, log2 = A6.train_model(model2_dict, dataloader_v2, datadict, device)

Now that you have trained our models, it is time to look at the training logs and plot the behaviour. This code is provided for you in the next two code blocks; you do not need to do any coding tasks for this part of the process, but you do need to answer a number of written questions. These will be in markdown cells after each code block. While we have logged loss curves (and you are welcome to explore them for your own interest and understanding), for simplicity we are going to concentrate our analysis here on the accuracy curves.

These plots will also be saved automatically into an `output` folder on your system, and you should submit them as part of your responses to the written questions (**Written Question 6**, 1 point for the first, 2 points for the second).

Note that the first set of plots are based on logs that are already produced by the provided code, whereas the second set of the plots are based on the class-wise accuracy plots you added code to support.

In [ ]:
if not os.path.exists('output/'):
    os.mkdir('output')

fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.set_ylim([0.6,1.0])
plt.plot(log1["train_acc"], 'r-')
plt.plot(log1["val_acc"], 'b-')
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy for Model V1')
plt.legend(['training', 'validation'])

ax = fig.add_subplot(1,2,2)
ax.set_ylim([0.6,1.0])
plt.plot(log2["train_acc"], 'r-')
plt.plot(log2["val_acc"], 'b-')
plt.title('Accuracy for Model V2')
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.legend(['training', 'validation'])

fig.savefig('output/model_acc.png', bbox_inches='tight', pad_inches=0)

**Written Question 2: Initial Network Loss Accuracy**: \[2 points\] 
As we saw above, our network started with extremely poor predictive ability, however the first datapoint for both versions is already well-above 60% accuracy for training accuracy. Why is that?

**Written Question 3: Initial Network Validation versus Loss Accuracy**: \[2 points\] 
Although we see variations between loss and validation accuracy at later epochs, there is a very large jump from the first training accuracy score to the first validation accuracy score. Why should we expect to see this jump?

**Written Question 4: Dataloader Differences**: \[3 points\] 
Compare the resultant behaviour of our two identical models trained with different dataloaders. What are the primary differences and similarities that you notice? What is the reason for these differences?


In [ ]:
class_acc1 = np.array(log1["class_acc"])
class_acc2 = np.array(log2["class_acc"])

colmat = [[0.5,0.5,0],[0.8,0,0],[0,0.8,0],[0.2,0.2,0.5],[0,0,0.8],[0.7,0,0.7],[0.3,0.5,0.2],[0.7,0.4,0.1]]

if not os.path.exists('output/'):
    os.mkdir('output')

fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.set_ylim([0.6,1.0])
for idx in range(datadict["num_classes"]):
    plt.plot(class_acc1[:,idx], color = colmat[idx], label = datadict["class_names"][idx])
plt.legend()
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.title('Class-wise Accuracy for Model V1')
    
ax = fig.add_subplot(1,2,2)
ax.set_ylim([0.6,1.0])
for idx in range(datadict["num_classes"]):
    plt.plot(class_acc2[:,idx], color = colmat[idx], label = datadict["class_names"][idx])
plt.legend()    
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.title('Class-wise Accuracy for Model V2')

fig.savefig('output/class_acc.png', bbox_inches='tight', pad_inches=0)


**Written Question 5: Class Accuracy**: \[3 points\] 
Examine the plots of class-wise accuracy shown above. Note that in both versions of our trained model we see the highest accuracy in the bear, dog, and cat classes, despite these classes having a smaller number of training samples than our other five classes. Since we do not include any normalization of loss for imbalanced classes, we might have expected the opposite behaviour. What explanation do you have for the pattern of performance we see?